In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import sys
from copy import deepcopy
from typing import List, Dict, Optional, Union, Callable
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import CapturedException

# Jupyter interactivity:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler, SavingOptions
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows

from pyphocorehelpers.print_helpers import CapturedException
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler

BATCH_DATE_TO_USE = '2023-10-04-GL' # used for filenames throught the notebook

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.Spike3D.pipeline.log


In [2]:
active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/nfs/turbo/umms-kdiba/Data')] # , Path(r'/home/halechr/FastData')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

# try to load an existing batch result:
global_batch_run = BatchRun.try_init_from_file(global_data_root_parent_path, active_global_batch_result_filename=active_global_batch_result_filename,
						skip_root_path_conversion=False, debug_print=debug_print) # on_needs_create_callback_fn=run_diba_batch

batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
batch_progress_df
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):  # more options can be specified also
    # display(batch_progress_df)
    # display(good_only_batch_progress_df)
    display(batch_progress_df)

Loading loaded session pickle file results : /nfs/turbo/umms-kdiba/Data/global_batch_result_2023-10-04-GL.pkl... done.
no difference between provided and internal paths.


,format_name,animal,exper_name,session_name,context,basedirs,status,errors,session_datetime,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-07 11:26:53,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,False,True
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.COMPLETED,None,2006-06-08 14:26:15,40,279,40,224,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.COMPLETED,None,2006-06-09 01:22:43,46,179,40,142,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
3,kdiba,gor01,one,2006-6-09_3-23-37,kdiba_gor01_one_2006-6-09_3-23-37,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.NOT_STARTED,None,2006-06-09 03:23:37,0,0,0,0,False,,,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,False,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.COMPLETED,None,2006-06-12 15:55:31,40,37,34,55,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,kdiba,pin01,one,fet11-04_21-20-3,kdiba_pin01_one_fet11-04_21-20-3,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,SessionBatchProgress.NOT_STARTED,None,2009-11-04 21:20:03,0,0,0,0,False,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet...,False,True
68,kdiba,pin01,one,redundant,kdiba_pin01_one_redundant,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/red...,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False
69,kdiba,pin01,one,showclus,kdiba_pin01_one_showclus,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/sho...,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False
70,kdiba,pin01,one,sleep,kdiba_pin01_one_sleep,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/sleep,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False


# Run Batch Executions/Computations

In [3]:
# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

included_session_batch_progress_df = batch_progress_df[np.isin(batch_progress_df['context'].values, included_session_contexts)]
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):  # more options can be specified also
    display(included_session_batch_progress_df)

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,session_datetime,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.COMPLETED,None,2006-06-08 14:26:15,40,279,40,224,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.COMPLETED,None,2006-06-09 01:22:43,46,179,40,142,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.COMPLETED,None,2006-06-12 15:55:31,40,37,34,55,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
6,kdiba,gor01,two,2006-6-07_16-40-19,kdiba_gor01_two_2006-6-07_16-40-19,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,SessionBatchProgress.COMPLETED,None,2006-06-07 16:40:19,42,212,40,333,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,True,True
8,kdiba,gor01,two,2006-6-08_21-16-25,kdiba_gor01_two_2006-6-08_21-16-25,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,SessionBatchProgress.COMPLETED,None,2006-06-08 21:16:25,40,45,40,62,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,kdiba,vvp01,two,2006-4-10_12-58-3,kdiba_vvp01_two_2006-4-10_12-58-3,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,SessionBatchProgress.COMPLETED,None,2006-04-10 12:58:03,40,47,42,16,True,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,True,True
52,kdiba,pin01,one,11-02_17-46-44,kdiba_pin01_one_11-02_17-46-44,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,SessionBatchProgress.COMPLETED,None,2009-11-02 17:46:44,54,60,66,66,True,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,True,True
53,kdiba,pin01,one,11-02_19-28-0,kdiba_pin01_one_11-02_19-28-0,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,SessionBatchProgress.COMPLETED,None,2009-11-02 19:28:00,56,48,50,19,True,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,True,True
54,kdiba,pin01,one,11-03_12-3-25,kdiba_pin01_one_11-03_12-3-25,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,SessionBatchProgress.COMPLETED,None,2009-11-03 12:03:25,50,10,46,6,True,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-...,True,True


# Execute Batch

In [ ]:
output_included_session_contexts, output_python_scripts, output_slurm_scripts = global_batch_run.generate_batch_slurm_jobs(included_session_contexts, Path('output/generated_slurm_scripts/').resolve(), 
                                                                                                                        use_separate_run_directories=True, should_force_reload_all=False, should_perform_figure_generation_to_file=False)

In [11]:
# %pdb on

# multiprocessing_kwargs = dict(use_multiprocessing=False, num_processes=1)
multiprocessing_kwargs = dict(use_multiprocessing=True, num_processes=7)

enable_full_pipeline_in_ram = False
# enable_full_pipeline_in_ram = True
## Error with enable_full_pipeline_in_ram=True:
 # delta_since_last_compute=datetime.timedelta(seconds=43, microseconds=466370), outputs_local={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl')}, outputs_global={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl'), 'hdf5': None}, across_session_results={'inst_fr_comps': None, 'curr_active_pipeline': <pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline.NeuropyPipeline object at 0x148e83474700>}))'. Reason: 'AttributeError("Can't pickle local object 'DataSessionFormatBaseRegisteredClass.build_default_filter_functions.<locals>.<dictcomp>.<lambda>'")'
    
# Whether to output figures:
should_perform_figure_generation_to_file=False
# should_perform_figure_generation_to_file=True

## Included Session Contexts:
# included_session_contexts = batch_progress_df[np.logical_and(batch_progress_df['has_user_replay_annotations'], batch_progress_df['is_ready'])]['context'].to_numpy().tolist()

# Only require sessions to have replay annotations:
# included_session_contexts = batch_progress_df[batch_progress_df['has_user_replay_annotations']]['context'].to_numpy().tolist()

# included_session_contexts = batch_progress_df['context'].to_numpy().tolist()[:4] # Only get the first 6
## Limit the contexts to run to the last N:
# included_session_contexts = included_session_contexts[3:6]

# ALL
included_session_contexts = included_session_contexts

# ## No filtering the sessions:
# included_session_contexts = None

if included_session_contexts is not None:
    print(f'len(included_session_contexts): {len(included_session_contexts)}')
else:
    print(f'included_session_contexts is None so all session contexts will be included.')

# included_session_contexts

# No recomputing at all:
# result_handler = BatchSessionCompletionHandler(force_reload_all=False,
#                                                 session_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=SavingOptions.NEVER),
#                                                 global_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=True, should_save=SavingOptions.NEVER),
#                                                 should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, should_generate_all_plots=True, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False,
#                                                 enable_full_pipeline_in_ram=enable_full_pipeline_in_ram,
#                                                 **multiprocessing_kwargs)

# No Reloading
result_handler = BatchSessionCompletionHandler(force_reload_all=False,
                                                session_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=True, should_save=SavingOptions.NEVER),
                                                global_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=True, should_save=SavingOptions.IF_CHANGED),
                                                should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, should_generate_all_plots=True, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False,
                                                enable_full_pipeline_in_ram=enable_full_pipeline_in_ram,
                                                **multiprocessing_kwargs)


# # Forced Reloading:
# result_handler = BatchSessionCompletionHandler(force_reload_all=True,
#                                                 session_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=SavingOptions.ALWAYS),
#                                                 global_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=SavingOptions.ALWAYS),
#                                                 should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE, force_global_recompute=True,
#                                                 **multiprocessing_kwargs)


active_post_run_callback_fn = result_handler.on_complete_success_execution_session
# active_post_run_callback_fn = _temp_on_complete_success_execution_session


## Specific Setup for 2023-09-28 Changes to LxC/SxC "refinements"
result_handler.extended_computations_include_includelist = ['pf_computation', 'pfdt_computation', 'firing_rate_trends',
                                                'pf_dt_sequential_surprise',
                                                'ratemap_peaks_prominence2d',
                                                'position_decoding', 
                                                'position_decoding_two_step', 
                                                'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping',
                                                'long_short_inst_spike_rate_groups',
                                                'long_short_endcap_analysis',
                                                ]


basic_local_computations = ['pf_computation', 'pfdt_computation', 'firing_rate_trends',
#                                                 'pf_dt_sequential_surprise',
                                                # 'ratemap_peaks_prominence2d',
                                                'position_decoding', 
                                                #'position_decoding_two_step', 
                                                ]
 
# result_handler.extended_computations_include_includelist = ['long_short_inst_spike_rate_groups']


# result_handler.enable_hdf5_output = True # output the HDF5 when done.
# result_handler.override_existing_frs_index_values = True
# result_handler.frs_index_inclusion_magnitude = 0.1

result_handler.enable_hdf5_output = True
result_handler.override_existing_frs_index_values = False


## Execute with the custom arguments.
global_batch_run.execute_all(force_reload=result_handler.force_reload_all, saving_mode=result_handler.saving_mode, skip_extended_batch_computations=True, post_run_callback_fn=active_post_run_callback_fn,
                             fail_on_exception=False, included_session_contexts=included_session_contexts,
                                                                                        **{'computation_functions_name_includelist': basic_local_computations,
                                                                                            'active_session_computation_configs': None,
                                                                                            'allow_processing_previously_completed': True}, **multiprocessing_kwargs) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 4m 39.8s

len(included_session_contexts): 15
Beginning processing with len(included_session_contexts): 15
build_batch_task_logger(module_name="gl0003.arc-ts.umich.edu.kdiba.gor01.one.2006-6-09_1-22-43"):build_batch_task_logger(module_name="gl0003.arc-ts.umich.edu.kdiba.gor01.one.2006-6-08_14-26-15"):

build_batch_task_logger(module_name="gl0003.arc-ts.umich.edu.kdiba.gor01.one.2006-6-12_15-55-31"):
build_batch_task_logger(module_name="gl0003.arc-ts.umich.edu.kdiba.gor01.two.2006-6-07_16-40-19"):	 Batch Task logger com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.gl0003.arc-ts.umich.edu.kdiba.gor01.one.2006-6-12_15-55-31 has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.gl0003.arc-ts.umich.edu.kdiba.gor01.one.2006-6-12_15-55-31.logbuild_batch_task_logger(module_name="gl0003.arc-ts.umich.edu.kdiba.gor01.two.2006-6-12_16-53-46"):

	 Batch Task logger com.PhoHale.PhoPy3DPositionAnalyis.Batc

/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/Analysis/Decoder/reconstruction.py:366: RuntimeWarning: divide by zero encountered in divide
  return C * np.exp(numerator/denominator)


done.
Loading pickled pipeline success: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle.pkl.
properties already present in pickled version. No need to save.
pipeline load success!
using provided computation_functions_name_includelist: ['pf_computation', 'pfdt_computation', 'firing_rate_trends', 'position_decoding']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
saving_mode.shouldSave == False, so not saving at the end of batch_load_session
on_complete_success_execution_session(curr_session_context: kdiba_gor01_one_2006-6-09_1-22-43, curr_session_bas

/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/Analysis/Decoder/reconstruction.py:366: RuntimeWarning: divide by zero encountered in divide
  return C * np.exp(numerator/denominator)


done.
Loading pickled pipeline success: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl.
properties already present in pickled version. No need to save.
pipeline load success!
using provided computation_functions_name_includelist: ['pf_computation', 'pfdt_computation', 'firing_rate_trends', 'position_decoding']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.done.

Loading pickled pipeline success: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/2006-6-09_22-24-40/loadedSessPickle.pkl.WARNING: skipping computation because overwrite_extant_results=False and active_computation_results[maze] already exists and is non-None

	 TODO: this will prevent recomputation even when the excludelist/includelist or co

...

a_field: LxC_aclus
	a_field_key: /kdiba/pin01/one/11-02_17-46-44/global_computations/inst_fr_comps/LxC_aclus
	 field not custom serializable! a_field_attr.type: <class 'numpy.ndarray'>.
a_field: SxC_aclus
	a_field_key: /kdiba/pin01/one/11-02_17-46-44/global_computations/inst_fr_comps/SxC_aclus
	 field not custom serializable! a_field_attr.type: <class 'numpy.ndarray'>.
a_field: Fig2_Replay_FR
	a_field_key: /kdiba/pin01/one/11-02_17-46-44/global_computations/inst_fr_comps/Fig2_Replay_FR
a_field: values
	a_field_key: /kdiba/pin01/one/11-02_17-46-44/global_computations/inst_fr_comps/Fig2/Replay/inst_FR_Bars/LxC_ReplayDeltaMinus/values
	 field not custom serializable! a_field_attr.type: <class 'numpy.ndarray'>.
a_field: LxC_aclus
	a_field_key: /kdiba/pin01/one/11-02_17-46-44/global_computations/inst_fr_comps/Fig2/Replay/inst_FR_Bars/LxC_ReplayDeltaMinus/LxC_aclus
	 field not custom serializable! a_field_attr.type: <class 'numpy.ndarray'>.
a_field: SxC_aclus
	a_field_key: /kdiba/pin01/one/1

/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/Analysis/Decoder/reconstruction.py:366: RuntimeWarning: divide by zero encountered in divide
  return C * np.exp(numerator/denominator)


	 doing specific instantaneous firing rate computation for context: kdiba_gor01_two_2006-6-09_22-24-40...
		 done (success).
"========================== END BATCH ==========================


two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (90848,)
	 done.
ERROR perform `batch_extended_computations` or saving GLOBAL COMPUTATION RESULTS for pipeline of curr_session_context: kdiba_pin01_one_fet11-01_12-58-54. error: !! 'pf1D_TwoStepDecoder' ::::: (<class 'KeyError'>, KeyError('pf1D_TwoStepDecoder'), <traceback object at 0x14b20629ac00>)
newly_computed_values: ['jonathan_firing_rate_analysis']. Saving global results...
global_computation_results_pickle_path: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results.pkl
Saving (file mode '/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results.pkl') saved session pickle file results : /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet

/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'fet11-01_12-58-54'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/MultiContextComputationFunctions/LongShortTrackComputations.py:266: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['firing_rates', 'is_neuron_active', 'active_aclus'], dtype='object')]

  self.rdf.rdf.to_hdf(file_path, key=f'{key}/rdf/df') # , format='table', data_columns=True Can't do 'table' format because `TypeError: Cannot serialize the column [firing_rates] because its data 

a_field: LxC_aclus
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/global_computations/inst_fr_comps/LxC_aclus
	 field not custom serializable! a_field_attr.type: <class 'numpy.ndarray'>.
a_field: SxC_aclus
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/global_computations/inst_fr_comps/SxC_aclus
	 field not custom serializable! a_field_attr.type: <class 'numpy.ndarray'>.
a_field: Fig2_Replay_FR
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/global_computations/inst_fr_comps/Fig2_Replay_FR
a_field: values
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/global_computations/inst_fr_comps/Fig2/Replay/inst_FR_Bars/LxC_ReplayDeltaMinus/values
	 field not custom serializable! a_field_attr.type: <class 'numpy.ndarray'>.
a_field: LxC_aclus
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/global_computations/inst_fr_comps/Fig2/Replay/inst_FR_Bars/LxC_ReplayDeltaMinus/LxC_aclus
	 field not custom serializable! a_field_attr.type: <class 'numpy.ndarray'>.
a_field: SxC_aclus
	a_field_key: /kd

In [12]:
# Save to pickle:
saveData(global_batch_result_file_path, global_batch_run) # Update the global batch run dictionary

# Save to HDF5
suffix = f'{BATCH_DATE_TO_USE}'
## Build Pickle Path:
file_path = global_data_root_parent_path.joinpath(f'global_batch_output_{suffix}.h5').resolve()
global_batch_run.to_hdf(file_path,'/')

Saving (file mode '/nfs/turbo/umms-kdiba/Data/global_batch_result_2023-10-04-GL.pkl') saved session pickle file results : /nfs/turbo/umms-kdiba/Data/global_batch_result_2023-10-04-GL.pkl... 	moving new output at '/nfs/turbo/umms-kdiba/Data/20231004134512-global_batch_result_2023-10-04-GL.pkltmp' -> to desired location: '/nfs/turbo/umms-kdiba/Data/global_batch_result_2023-10-04-GL.pkl'
done.
a_field: long_laps
	a_field_key: /kdiba/gor01/one/2006-6-08_14-26-15/batch_result/long_laps
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: long_replays
	a_field_key: /kdiba/gor01/one/2006-6-08_14-26-15/batch_result/long_replays
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: short_laps
	a_field_key: /kdiba/gor01/one/2006-6-08_14-26-15/batch_result/short_laps
	 field is serializable! Calling a_value.to_hdf(...)...


/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '2006-6-08_14-26-15'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '2006-6-09_1-22-43'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


a_field: short_replays
	a_field_key: /kdiba/gor01/one/2006-6-08_14-26-15/batch_result/short_replays
	 field is serializable! Calling a_value.to_hdf(...)...
an_attribute_field: long_epoch_name
an_attribute_field: short_epoch_name
a_field: long_laps
	a_field_key: /kdiba/gor01/one/2006-6-09_1-22-43/batch_result/long_laps
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: long_replays
	a_field_key: /kdiba/gor01/one/2006-6-09_1-22-43/batch_result/long_replays
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: short_laps
	a_field_key: /kdiba/gor01/one/2006-6-09_1-22-43/batch_result/short_laps
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: short_replays
	a_field_key: /kdiba/gor01/one/2006-6-09_1-22-43/batch_result/short_replays
	 field is serializable! Calling a_value.to_hdf(...)...
an_attribute_field: long_epoch_name
an_attribute_field: short_epoch_name
a_field: long_laps
	a_field_key: /kdiba/gor01/one/2006-6-12_15-55-31/batch_result/long_laps
	 

/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '2006-6-12_15-55-31'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


a_field: short_replays
	a_field_key: /kdiba/gor01/one/2006-6-12_15-55-31/batch_result/short_replays
	 field is serializable! Calling a_value.to_hdf(...)...
an_attribute_field: long_epoch_name
an_attribute_field: short_epoch_name
a_field: long_laps
	a_field_key: /kdiba/gor01/two/2006-6-07_16-40-19/batch_result/long_laps
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: long_replays
	a_field_key: /kdiba/gor01/two/2006-6-07_16-40-19/batch_result/long_replays
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: short_laps
	a_field_key: /kdiba/gor01/two/2006-6-07_16-40-19/batch_result/short_laps
	 field is serializable! Calling a_value.to_hdf(...)...


...

/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '11-02_17-46-44'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '11-02_19-28-0'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


an_attribute_field: long_epoch_name
an_attribute_field: short_epoch_name
a_field: long_laps
	a_field_key: /kdiba/pin01/one/11-02_19-28-0/batch_result/long_laps
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: long_replays
	a_field_key: /kdiba/pin01/one/11-02_19-28-0/batch_result/long_replays
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: short_laps
	a_field_key: /kdiba/pin01/one/11-02_19-28-0/batch_result/short_laps
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: short_replays
	a_field_key: /kdiba/pin01/one/11-02_19-28-0/batch_result/short_replays
	 field is serializable! Calling a_value.to_hdf(...)...
an_attribute_field: long_epoch_name
an_attribute_field: short_epoch_name
a_field: long_laps
	a_field_key: /kdiba/pin01/one/11-03_12-3-25/batch_result/long_laps
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: long_replays
	a_field_key: /kdiba/pin01/one/11-03_12-3-25/batch_result/long_replays
	 field is serializable! Calli

/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '11-03_12-3-25'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


an_attribute_field: long_epoch_name
an_attribute_field: short_epoch_name
a_field: long_laps
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/batch_result/long_laps
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: long_replays
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/batch_result/long_replays
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: short_laps
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/batch_result/short_laps
	 field is serializable! Calling a_value.to_hdf(...)...
a_field: short_replays
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/batch_result/short_replays
	 field is serializable! Calling a_value.to_hdf(...)...
an_attribute_field: long_epoch_name
an_attribute_field: short_epoch_name
done outputting HDF file.


/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'fet11-01_12-58-54'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [13]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,session_datetime,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.COMPLETED,None,2006-06-08 14:26:15,40,279,40,224,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.COMPLETED,None,2006-06-09 01:22:43,46,179,40,142,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,SessionBatchProgress.COMPLETED,None,2006-06-12 15:55:31,40,37,34,55,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/200...,True,True
6,kdiba,gor01,two,2006-6-07_16-40-19,kdiba_gor01_two_2006-6-07_16-40-19,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,SessionBatchProgress.COMPLETED,None,2006-06-07 16:40:19,42,212,40,333,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,True,True
8,kdiba,gor01,two,2006-6-08_21-16-25,kdiba_gor01_two_2006-6-08_21-16-25,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,SessionBatchProgress.COMPLETED,None,2006-06-08 21:16:25,40,45,40,62,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,True,True
9,kdiba,gor01,two,2006-6-09_22-24-40,kdiba_gor01_two_2006-6-09_22-24-40,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,SessionBatchProgress.COMPLETED,None,2006-06-09 22:24:40,51,106,43,406,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,True,True
10,kdiba,gor01,two,2006-6-12_16-53-46,kdiba_gor01_two_2006-6-12_16-53-46,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,SessionBatchProgress.COMPLETED,None,2006-06-12 16:53:46,41,59,40,48,True,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/200...,True,True
12,kdiba,vvp01,one,2006-4-09_17-29-30,kdiba_vvp01_one_2006-4-09_17-29-30,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/200...,SessionBatchProgress.COMPLETED,None,2006-04-09 17:29:30,51,45,42,62,True,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/200...,True,True
13,kdiba,vvp01,one,2006-4-10_12-25-50,kdiba_vvp01_one_2006-4-10_12-25-50,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/200...,SessionBatchProgress.COMPLETED,None,2006-04-10 12:25:50,50,22,42,17,True,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/200...,True,True
31,kdiba,vvp01,two,2006-4-09_16-40-54,kdiba_vvp01_two_2006-4-09_16-40-54,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,SessionBatchProgress.COMPLETED,None,2006-04-09 16:40:54,48,27,50,20,True,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/200...,True,True


# Post-Hoc Processing

In [ ]:
included_session_batch_progress_df

# Across Sessions After Batching Complete

In [14]:
from datetime import datetime, timedelta
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder

good_session_concrete_folders = [ConcreteSessionFolder(a_context, a_basedir) for a_context, a_basedir in zip(list(good_only_batch_progress_df.context.values), list(good_only_batch_progress_df.basedirs.values))]
good_session_concrete_folders

# good_only_batch_progress_df.batch_results
# included_h5_paths = [get_file_str_if_file_exists(v.joinpath('output','pipeline_results.h5').resolve()) for v in list(good_only_batch_progress_df.basedirs.values)]
# included_h5_paths = [a_dir.joinpath('output','pipeline_results.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']]
included_h5_paths = [get_file_str_if_file_exists(v.pipeline_results_h5) for v in good_session_concrete_folders]

## Extract `across_sessions_instantaneous_fr_dict` from the computation outputs

In [15]:
# Get only the sessions with non-None results
sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]

# list(global_batch_run.session_batch_outputs.keys())

# Somewhere in there there are `InstantaneousSpikeRateGroupsComputation` results to extract
across_sessions_instantaneous_fr_dict = {} # InstantaneousSpikeRateGroupsComputation

# good_session_batch_outputs = global_batch_run.session_batch_outputs

sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
good_session_batch_outputs = {a_ctxt:a_result for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None}

for a_ctxt, a_result in good_session_batch_outputs.items():
    if a_result is not None:
        # a_good_result = a_result.__dict__.get('across_sessions_batch_results', {}).get('inst_fr_comps', None)
        a_good_result = a_result.across_session_results.get('inst_fr_comps', None)
        if a_good_result is not None:
            across_sessions_instantaneous_fr_dict[a_ctxt] = a_good_result
            # print(a_result['across_sessions_batch_results']['inst_fr_comps'])
            
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
                                                 inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')

# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

num_sessions: 15
global_batch_result_inst_fr_file_path: /nfs/turbo/umms-kdiba/Data/across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl
Saving (file mode '/nfs/turbo/umms-kdiba/Data/across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl') saved session pickle file results : /nfs/turbo/umms-kdiba/Data/across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl... 	moving new output at '/nfs/turbo/umms-kdiba/Data/20231004134528-across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkltmp' -> to desired location: '/nfs/turbo/umms-kdiba/Data/across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl'
done.


In [26]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionTables

# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True, )

neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths, override_output_parent_path=global_data_root_parent_path, output_path_suffix=f'{BATCH_DATE_TO_USE}')

failed for file path: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-08_14-26-15/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-08_14-26-15/neuron_identities/table``. Skipping.
failed for file path: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-09_1-22-43/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-09_1-22-43/neuron_identities/table``. Skipping.
failed for file path: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table``. Skipping.
failed for file path: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two

In [20]:
long_short_fr_indicies_analysis_table

,format_name,animal,exper_name,session_name,index,neuron_uid,session_uid,aclu,x_frs_index,y_frs_index,session_datetime
0,kdiba,gor01,one,2006-6-08_14-26-15,0,kdiba|gor01|one|2006-6-08_14-26-15|2,kdiba|gor01|one|2006-6-08_14-26-15,2,0.947513,1.000000,2006-06-08 14:26:15
1,kdiba,gor01,one,2006-6-08_14-26-15,1,kdiba|gor01|one|2006-6-08_14-26-15|3,kdiba|gor01|one|2006-6-08_14-26-15,3,-0.855006,-0.544194,2006-06-08 14:26:15
2,kdiba,gor01,one,2006-6-08_14-26-15,2,kdiba|gor01|one|2006-6-08_14-26-15|4,kdiba|gor01|one|2006-6-08_14-26-15,4,-0.972594,-1.000000,2006-06-08 14:26:15
3,kdiba,gor01,one,2006-6-08_14-26-15,3,kdiba|gor01|one|2006-6-08_14-26-15|5,kdiba|gor01|one|2006-6-08_14-26-15,5,0.133418,-0.032454,2006-06-08 14:26:15
4,kdiba,gor01,one,2006-6-08_14-26-15,4,kdiba|gor01|one|2006-6-08_14-26-15|6,kdiba|gor01|one|2006-6-08_14-26-15,6,0.433462,0.454120,2006-06-08 14:26:15
...,...,...,...,...,...,...,...,...,...,...,...
882,kdiba,pin01,one,fet11-01_12-58-54,26,kdiba|pin01|one|fet11-01_12-58-54|28,kdiba|pin01|one|fet11-01_12-58-54,28,-0.567445,-0.172839,2009-11-01 12:58:54
883,kdiba,pin01,one,fet11-01_12-58-54,27,kdiba|pin01|one|fet11-01_12-58-54|29,kdiba|pin01|one|fet11-01_12-58-54,29,-0.176580,0.089972,2009-11-01 12:58:54
884,kdiba,pin01,one,fet11-01_12-58-54,28,kdiba|pin01|one|fet11-01_12-58-54|30,kdiba|pin01|one|fet11-01_12-58-54,30,-0.219618,-0.072261,2009-11-01 12:58:54
885,kdiba,pin01,one,fet11-01_12-58-54,29,kdiba|pin01|one|fet11-01_12-58-54|31,kdiba|pin01|one|fet11-01_12-58-54,31,0.314634,0.160301,2009-11-01 12:58:54


In [21]:
neuron_replay_stats_table

,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,...,neuron_type,aclu,custom_frs_index,is_rate_extrema,is_refined_exclusive,is_refined_LxC,is_refined_SxC,session_uid,neuron_uid,session_datetime
0,kdiba,gor01,one,2006-6-08_14-26-15,212.160000,NaN,LEFT_ONLY,0.419960,0.114724,-0.305237,...,pyr,2,0.235886,False,False,False,False,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|2,2006-06-08 14:26:15
1,kdiba,gor01,one,2006-6-08_14-26-15,136.160000,198.160000,SHARED,0.275226,1.227759,0.952532,...,pyr,3,-0.683602,True,False,False,False,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|3,2006-06-08 14:26:15
2,kdiba,gor01,one,2006-6-08_14-26-15,NaN,178.160000,RIGHT_ONLY,0.117397,0.834271,0.716874,...,pyr,4,-0.981692,True,True,False,True,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|4,2006-06-08 14:26:15
3,kdiba,gor01,one,2006-6-08_14-26-15,98.160000,202.160000,SHARED,2.090985,1.844885,-0.246099,...,pyr,5,-0.057305,False,False,False,False,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|5,2006-06-08 14:26:15
4,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,SHARED,26.129319,12.892497,-13.236821,...,intr,6,0.102215,False,False,False,False,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|6,2006-06-08 14:26:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,kdiba,pin01,one,fet11-01_12-58-54,179.403791,177.403791,SHARED,0.892852,1.027642,0.134789,...,pyr,28,-0.148122,False,False,False,False,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|28,2009-11-01 12:58:54
883,kdiba,pin01,one,fet11-01_12-58-54,59.403791,107.403791,SHARED,0.837484,0.766268,-0.071216,...,pyr,29,0.041598,False,False,False,False,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|29,2009-11-01 12:58:54
884,kdiba,pin01,one,fet11-01_12-58-54,119.403791,127.403791,SHARED,0.712330,0.577649,-0.134681,...,pyr,30,-0.026850,False,False,False,False,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|30,2009-11-01 12:58:54
885,kdiba,pin01,one,fet11-01_12-58-54,217.403791,33.403791,SHARED,0.902182,1.016473,0.114291,...,pyr,31,0.039185,False,False,False,False,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|31,2009-11-01 12:58:54


In [17]:
AcrossSessionTables.save_out_to_combined_file(included_session_contexts, included_h5_paths)

failed for file path: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-08_14-26-15/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-08_14-26-15/neuron_identities/table``. Skipping.
failed for file path: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-09_1-22-43/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-09_1-22-43/neuron_identities/table``. Skipping.
failed for file path: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table``. Skipping.
failed for file path: /nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two

In [18]:
# target_dir = Path('output/across_session_results/2023-09-29').resolve()
# target_dir = Path('/home/halechr/cloud/turbo/Pho/Output/across_session_results/2023-09-29').resolve()
target_dir = Path('/home/halechr/cloud/turbo/Pho/Output/across_session_results/2023-10-04').resolve()
copy_dict = ConcreteSessionFolder.backup_output_files(good_session_concrete_folders, target_dir=target_dir)
copy_dict

{PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5'): PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/across_session_results/2023-10-04/kdiba_gor01_one_2006-6-08_14-26-15_pipeline_results.h5'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl'): PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/across_session_results/2023-10-04/kdiba_gor01_one_2006-6-08_14-26-15_loadedSessPickle.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl'): PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/across_session_results/2023-10-04/kdiba_gor01_one_2006-6-08_14-26-15_global_computation_results.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5'): PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/across_session_results/2023-10-04/kdiba_gor01_one_2006-6-09_1-22-43_pipeline_results.h5'),
 PosixPath('/nfs/turbo/um

In [19]:
moved_files_dict_h5_files = copy_movedict(copy_dict)

copying "/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5"
		 -> "/nfs/turbo/umms-kdiba/Pho/Output/across_session_results/2023-10-04/kdiba_gor01_one_2006-6-08_14-26-15_pipeline_results.h5"...
done.
copying "/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl"
		 -> "/nfs/turbo/umms-kdiba/Pho/Output/across_session_results/2023-10-04/kdiba_gor01_one_2006-6-08_14-26-15_loadedSessPickle.pkl"...
done.
copying "/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl"
		 -> "/nfs/turbo/umms-kdiba/Pho/Output/across_session_results/2023-10-04/kdiba_gor01_one_2006-6-08_14-26-15_global_computation_results.pkl"...
done.
copying "/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5"
		 -> "/nfs/turbo/umms-kdiba/Pho/Output/across_session_results/2023-10-04/kdiba_gor01_one_2006-6-09_1-22-43_pipeline_results.h5"...
done.
copying "/nfs/turbo/umms-kdiba/Da

In [ ]:
check_output_h5_files(included_h5_paths)

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import copy_files_in_filelist_to_dest

copy_files_in_filelist_to_dest(filelist_text_file="/nfs/turbo/umms-kdiba/Data/fileList_GreatLakes_HDF5_2023-09-29-GL.txt", target_directory=Path('output/extracted_hdf5_files/').resolve())

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import build_output_filelists

included_session_basedirs = included_session_batch_progress_df['basedirs']
output_filelist_transfer_dict = build_output_filelists(filelist_save_parent_path=global_data_root_parent_path, included_session_basedirs=included_session_basedirs, BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, dest_computer_name='LabWorkstation')
# output_filelist_transfer_dict = build_output_filelists(filelist_save_parent_path=global_data_root_parent_path, included_session_basedirs=included_session_basedirs, BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, dest_computer_name='Apogee')
output_filelist_transfer_dict

In [ ]:
# np.sum(neuron_replay_stats_table['is_refined_LxC'])
# np.isnan(neuron_replay_stats_table['is_refined_LxC'])

## 2023-10-04 - Explore Outputs

In [ ]:
# Options
session_identifier_key: str = 'session_name'
# session_identifier_key: str = 'session_datetime'

## !IMPORTANT! Count of the fields of interest using .value_counts(...) and converting to an explicit pd.DataFrame:
# _out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
# _out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'count']
_out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership','is_refined_LxC', 'is_refined_SxC'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
_out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'is_refined_LxC', 'is_refined_SxC', 'count']
_out_value_counts_df

In [ ]:
## Find the time of the first session for each animal:
first_session_time  = _out_value_counts_df.groupby(['animal']).agg(session_datetime_first=('session_datetime', 'first')).reset_index()

## Subtract this initial time from all of the 'session_datetime' entries for each animal:
# Merge the first session time back into the original DataFrame
merged_df = pd.merge(_out_value_counts_df, first_session_time, on='animal')

# Subtract this initial time from all of the 'session_datetime' entries for each animal
merged_df['time_since_first_session'] = merged_df['session_datetime'] - merged_df['session_datetime_first']

merged_df

In [ ]:
import matplotlib.pyplot as plt

point_size = 8
df = _out_value_counts_df.copy()
animals = df['animal'].unique()
track_memberships = df['track_membership'].unique()

fig, axes = plt.subplots(1, len(animals), figsize=(15, 5))

for i, animal in enumerate(animals):
	ax = axes[i]
	subset_df = df[df['animal'] == animal]
	
	for track_membership in track_memberships:
		track_subset_df = subset_df[subset_df['track_membership'] == track_membership]
		ax.plot(track_subset_df['session_datetime'], track_subset_df['count'], label=f'Track: {track_membership}')
		ax.scatter(track_subset_df['session_datetime'], track_subset_df['count'], s=point_size)
		
	ax.set_title(f'Animal: {animal}')
	ax.set_xlabel('Session Datetime')
	ax.set_ylabel('Count')
	ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
_out_value_counts_df

In [ ]:


## See if the number of cells decreases over re-exposures to the track
df = _out_value_counts_df[_out_value_counts_df['animal'] == 'gor01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'pin01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'vvp01']

# Sort by column: 'session_datetime' (ascending)
df = df.sort_values(['session_datetime'])

'LEFT_ONLY'

# df.to_clipboard(index=False)
df

In [ ]:
## Get the number of cells in each session of the animal:
num_LxCs = df[df['track_membership'] == 'LEFT_ONLY']['count'].to_numpy()
num_Shared = df[df['track_membership'] == 'SHARED']['count'].to_numpy()
num_SxCs = df[df['track_membership'] == 'RIGHT_ONLY']['count'].to_numpy()

num_TotalCs = num_LxCs + num_Shared + num_SxCs
num_TotalCs

In [ ]:
# The only safe point to align each session to is the switchpoint (the delta):


In [ ]:
# Each session can be expressed in terms of time from the start of the first session.


In [ ]:
df.plot()


# AcrossSessionsVisualizations

In [ ]:

from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=0)


In [ ]:

from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=0)

In [ ]:
included_h5_paths = [a_dir.joinpath('output','pipeline_results.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']]
included_global_computation_h5_paths = [a_dir.joinpath('output','global_computations.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']] 
included_h5_paths

## Extract output files from all completed sessions:

In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import convert_filelist_to_new_parent
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import save_filelist_to_text_file


# Save output filelist:

# '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5'

# kdiba_vvp01_two_2006-4-10_12-58-3
# 	outputs_local ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/loadedSessPickle.pkl')}
# 	outputs_global ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/global_computation_results.pkl'), 'hdf5': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/pipeline_results.h5')}
session_identifiers, pkl_output_paths, hdf5_output_paths = global_batch_run.build_output_files_lists()

h5_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_HDF5_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_HDF5_savepath = save_filelist_to_text_file(hdf5_output_paths, h5_filelist_path)

pkls_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_pkls_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_pkls_savepath = save_filelist_to_text_file(pkl_output_paths, pkls_filelist_path)

# source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
source_parent_path = Path(r'/nfs/turbo/umms-kdiba/Data')
dest_parent_path = Path(r'/~/W/Data/')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_source = hdf5_output_paths
filelist_dest_paths = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest_paths

dest_Apogee_h5_filelist_path = global_data_root_parent_path.joinpath(f'dest_fileList_Apogee_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, dest_filelist_savepath = save_filelist_to_text_file(filelist_dest_paths, dest_Apogee_h5_filelist_path)

# OLD

## 2023-07-14 - Load Saved across-sessions-data and testing Batch-computed inst_firing_rates:

In [11]:
# from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update
# from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
# from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import list_of_dicts_to_dict_of_lists
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations


In [12]:

## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-21.pkl'
# inst_fr_output_filename=f'across_session_result_handler_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-08-09_Test.pkl'
inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

global_batch_result_inst_fr_file_path: /nfs/turbo/umms-kdiba/Data/across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl
Loading loaded session pickle file results : /nfs/turbo/umms-kdiba/Data/across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl... done.
num_sessions: 15
num_sessions: 15


In [ ]:
across_sessions_instantaneous_frs_list

In [13]:
## Hacks the `PaperFigureTwo` and `InstantaneousSpikeRateGroupsComputation` 
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False)


num_sessions: 15
register_output_file(output_path: /home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2023-10-04/kdiba/15_2_inst_FR_bar_graphs.png, ...)
	 saved /home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2023-10-04/kdiba/15_2_inst_FR_bar_graphs.png
register_output_file(output_path: /home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2023-10-04/kdiba/15_2_inst_FR_bar_graphs.png, ...)
	 saved /home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2023-10-04/kdiba/15_2_inst_FR_bar_graphs.png
save_figure()!


In [ ]:
# _out_aggregate_fig_2


In [ ]:
across_session_inst_fr_computation.LxC_scatter_props
across_session_inst_fr_computation.SxC_scatter_props

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, InstantaneousFiringRatesDataframeAccessor, InstantaneousSpikeRateGroupsComputation, trackMembershipTypesEnum, trackExclusiveToMembershipTypeDict, trackExclusiveToMembershipTypeReverseDict

## Specify the output file:
common_file_path = Path('output/test_across_session_scatter_plot_new.h5')
print(f'common_file_path: {common_file_path}')
InstantaneousFiringRatesDataframeAccessor.add_results_to_inst_fr_results_table(curr_active_pipeline, common_file_path, file_mode='a')

In [ ]:
# Build the unique scatter plot dictionaries:
across_session_contexts = list(across_sessions_instantaneous_fr_dict.keys())
unique_animals = IdentifyingContext.find_unique_values(across_session_contexts)['animal'] # {'gor01', 'pin01', 'vvp01'}
# Get number of animals to plot
marker_list = [(5, i) for i in np.arange(len(unique_animals))] # [(5, 0), (5, 1), (5, 2)]
scatter_props = [{'marker': mkr} for mkr in marker_list]  # Example, you should provide your own scatter properties
scatter_props_dict = dict(zip(unique_animals, scatter_props))
# {'pin01': {'marker': (5, 0)},
#  'gor01': {'marker': (5, 1)},
#  'vvp01': {'marker': (5, 2)}}
scatter_props_dict

# Pass a function that will return a set of kwargs for a given context
def _return_scatter_props_fn(ctxt: IdentifyingContext):
	""" captures `scatter_props_dict` """
	animal_id = str(ctxt.animal)
	return scatter_props_dict[animal_id]
	


In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation # the result loaded from the file
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
# _out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)
_out_fig_2.scatter_props_fn = _return_scatter_props_fn

In [ ]:
LxC_aclus = _out_fig_2.computation_result.LxC_aclus
SxC_aclus = _out_fig_2.computation_result.SxC_aclus

LxC_aclus

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import FigureOutputManager, FigureOutputLocation, ContextToPathMode

registered_output_files = {}

def output_figure(final_context: IdentifyingContext, fig, write_vector_format:bool=False, write_png:bool=True, debug_print=True):
    """ outputs the figure using the provided context. """
    from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_and_write_to_file
    def register_output_file(output_path, output_metadata=None):
        """ registers a new output file for the pipeline """
        print(f'register_output_file(output_path: {output_path}, ...)')
        registered_output_files[output_path] = output_metadata or {}

    fig_out_man = FigureOutputManager(figure_output_location=FigureOutputLocation.DAILY_PROGRAMMATIC_OUTPUT_FOLDER, context_to_path_mode=ContextToPathMode.HIERARCHY_UNIQUE)
    active_out_figure_paths = build_and_write_to_file(fig, final_context, fig_out_man, write_vector_format=write_vector_format, write_png=write_png, register_output_file_fn=register_output_file)
    return active_out_figure_paths, final_context


# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = output_figure

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=global_multi_session_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)", save_figure=True)
	
_out_fig_2.perform_save()

### Single Session testing:


In [ ]:
_test_out = global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, computation_functions_name_includelist =['_perform_baseline_placefield_computation'], active_session_computation_configs=None) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)
_test_out

# global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, **{'computation_functions_name_includelist': ['_perform_baseline_placefield_computation'], 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 23.5s

In [ ]:

full_good_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is None]
bad_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is not None]
full_good_dirs
bad_dirs

In [ ]:
global_batch_run.session_batch_status

In [ ]:
global_batch_run.session_batch_status
global_batch_run.session_batch_errors

## Get good sessions for use in the specific session processing notebook:

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

In [ ]:
## Get the list of sessions that are completely ready to process:
full_good_ready_to_process_sessions = list(good_only_batch_progress_df['context'].to_numpy())
full_good_ready_to_process_sessions
# Get good sessions for use in the specific session processing notebook:

In [ ]:
run["good_sessions_list"].extend(full_good_ready_to_process_sessions)

In [ ]:
run.stop()
project.stop()

In [ ]:

print(",\n".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # List definitions

# [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
# IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')]

In [ ]:
print("\ncurr_context = ".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # Line definitions

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')

In [ ]:
good_only_batch_progress_df

In [ ]:
from datetime import datetime

# datetime object containing current date and time
save_time = datetime.now()
 
print("save_time =", save_time)

# dd/mm/YY H:M:S
dt_string = save_time.strftime("%Y-%m-%d_%I-%M%p")
print("date and time =", dt_string)

In [ ]:
## Get output file paths:
completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'outputs/global_computation_results.pkl'

full_good_ready_to_process_session_paths = list(good_only_batch_progress_df['basedirs'].to_numpy())
session_paths_output_folders = [sess_path.joinpath('outputs').resolve() for sess_path in full_good_ready_to_process_session_paths]

completed_pipeline_file_paths = [sess_path.joinpath(completed_pipeline_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths = [sess_path.joinpath(completed_global_computations_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths

In [ ]:
# Countable Additivity 
# Any countable collections of points is size 0

